## ReAct Agent

ReAct框架的核心在于将推理和行动紧密结合起来，其设计哲学是：在动态和不确定性的环境中，有效的决策需要持续的学习和适应，以及快速将推理转化为行动的能力，即形成有效的观察-思考-行动-再观察的循环。

<img src="../assets/react.png" alt="examples" style="zoom:30%;" /> 

In [1]:
%pip install google-search-results

Note: you may need to restart the kernel to use updated packages.


In [3]:
# 加载api key 
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("SERPAPI_API_KEY")

In [4]:
from langchain import hub
prompt = hub.pull("hwchase17/react")
print(prompt.template)

/Users/jiatianyu/miniconda3/envs/llm_env/lib/python3.10/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [5]:
from langchain_openai import OpenAI
from langchain.tools import Tool
from langchain_community.utilities import SerpAPIWrapper

llm = OpenAI(model="deepseek-chat",
             api_key=api_key,
             base_url="https://api.deepseek.com/beta")

search = SerpAPIWrapper()

tools = [
    Tool(
        name = "Google Search",
        func=search.run,
        description="当大模型没有相关知识时，用于搜索知识"
    ),
]

In [8]:
from langchain.agents import AgentExecutor
from langchain.agents import create_react_agent

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "当前北京时间是多少?"})



> Entering new AgentExecutor chain...
 我需要获取当前的北京时间，可以通过网络搜索来获取准确的时间信息。
Action: Google Search
Action Input: "当前北京时间"10:43 AM我通过搜索获取了当前的北京时间。
Final Answer: 当前北京时间是上午10点43分。

> Finished chain.


{'input': '当前北京时间是多少?', 'output': '当前北京时间是上午10点43分。'}